# Automate the degree of consolidation using Python


In [43]:
#load in the path to the Plaxis 
import imp
plaxis_path = r'C:\Program Files\Plaxis\PLAXIS 2D'
found_module = imp.find_module('plxscripting', [plaxis_path])
plxscripting = imp.load_module('plxscripting', *found_module)
from plxscripting.easy import *
import os
import numpy as np
import pandas as pd

In [44]:
port = 10001
s_o, g_o = new_server('localhost', port)
s_o.open(r'C:\Users\dazhong.li\Desktop\New folder (4)\MCC_AxSC_UPMW_r02.p2dx')

'OK'

In [35]:
def get_result_epwp(g_o,Phase):
    '''
    This function returns nodal excess pore water perssures at a certain phase
    Parameter: 
    g_o: Plaxis output object
    Phase: Phase of interested,example 'g_o.Phase_6'
    '''
    x = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.X, "node"))
    y = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.Y, "node"))
    epwp =np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.PExcess, "node"))
    df = pd.DataFrame(dict(x=[],y=[],epwp=[]))
    df.x = x; df.y = y; df.epwp = epwp
    return df 

In [38]:
df_Phase_6 = get_result_epwp(g_o, g_o.Phase_6)
df_Phase_7 = get_result_epwp(g_o, g_o.Phase_7)

In [42]:
df_Phase_6

,x,y,epwp
0,0.000000,4.143855,-0.010774
1,0.000000,3.529710,-0.021227
2,0.447355,3.934614,-0.014335
3,0.000000,3.836722,-0.016001
4,0.223677,3.732136,-0.017781
5,0.223677,4.039228,-0.012555
6,0.000000,3.990274,-0.013391
7,0.000000,3.683201,-0.018611
8,0.111839,3.630916,-0.019501
9,0.335516,3.833369,-0.016060


# Calculate the Avarege Excess Pore Water Pressure 


$$u_{avg} = \frac{1}{V} \iiint_V u(r,z)r\cdot d\theta dzdr = \frac{1}{V} \iint_V u(r,z) 2\pi rdrdz =  \frac{2\pi}{V} \iint_V u(r,z) rdrdz$$

The discretization of the integration gives:

$$ u_{avg} = \frac{2}{ (R^2 - r_o^2)H} \sum_i^m\sum_j^n (u_{ij} r_j \Delta r_{ij} \Delta z_{ij})$$

where n - number of discretization, R - radius of soil column and $r_o$ - radius of well

As we are using regular mesh, $\Delta z_{ij}$ and $\Delta r_{ij}$ are constant, the discrete form of integration could be further simplified as:

$$ u_{avg} = \frac{2\Delta r \Delta z}{ (R^2 - r_o^2)H} \sum_i^m\sum_j^n u_{ij} r_j  = C \sum_i^m\sum_j^n u_{ij} r_j$$

where $C$ is a constant once discretization is estabilished and $ C = \frac{2\Delta r \Delta z}{ (R^2 - r_o^2)H}$

In deriving the averge degree of consolidation, we will overlay a structured $m\times n$ mesh on top of the the existing output from Plaxis. The nodal values will be calculated using interpolation. It's expected that as n gets larger, the calculated average u will converge to a constant.